# 第15章 字符串

## 15.1 单词

需求：为文档中包含的单词生成一个列表，统计出现频率最高的单词。读取詹姆斯一世钦定版《圣经》中出现频率最高的21个单词，按数值递减的次序排序。

实现：代码使用dict字典表实现。

In [1]:
from collections import defaultdict

M = defaultdict(int)

with open("./data/kjv.txt", encoding="utf8") as f:
    lines = f.readlines()
    for line in lines:
        words = line.split()
        for word in words:
            M[word] += 1

In [3]:
result = sorted(M.items(), key=lambda x: -x[1])
result = result[:21]

rows = 7
cols = 3

for i in range(7):
    for j in range(3):
        index = j * rows + i
        kv = result[index]
        print("{:<5} {:<6}".format(kv[0], kv[1]), end="  ")
    print()

the   62051   shall 9759    they  6893    
and   38572   he    9509    be    6717    
of    34393   unto  8933    is    6696    
to    13367   I     8707    with  5951    
And   12739   his   8363    not   5840    
that  12454   a     7945    all   5238    
in    12167   for   7141    thou  4629    


通过本节示例，表示单词集合有两种主要方法：
- 平衡搜索树将字符串看作不可分割的对象进行操作，set和map大部分实现都使用这种结构。
- 散列需要深入字符串内部，计算散列函数，并将关键字分散到一个较大的表中。该方法的平均速度很块。

## 15.2 短语

需求：给定一个文本文件作为输入，查找其中最长的重复子字符串。

实现：使用“后缀数组”的数据结构，如果某个长字符串在数组c中出现两次，那么它将出现在两个不同的后缀中。

In [12]:
# 构建后缀数组
def build_suffix_array(s):
    suffixes = [(s[i:], i) for i in range(len(s))]
    suffixes.sort()
    return [suffix[1] for suffix in suffixes]

In [13]:
# 比较函数
def comlen(s, suffix_array):
    lcp_array = [0] * len(s)
    inv_suffix_array = [0] * len(s)
    for i in range(len(s)):
        inv_suffix_array[suffix_array[i]] = i
    lcp = 0
    for i in range(len(s)):
        if inv_suffix_array[i] == len(s) - 1:
            lcp = 0
            continue
        j = suffix_array[inv_suffix_array[i] + 1]
        while i + lcp < len(s) and j + lcp < len(s) and s[i + lcp] == s[j + lcp]:
            lcp += 1
        lcp_array[inv_suffix_array[i] + 1] = lcp
        if lcp > 0:
            lcp -= 1
    return lcp_array

In [14]:
def longest_common_substring(s):
    suffix_array = build_suffix_array(s)
    lcp_array = comlen(s, suffix_array)
    result = dict()
    for i in range(1, len(s)):
        strlen = lcp_array[i]
        index = i
        substr = s[suffix_array[index]:suffix_array[index] + strlen]
        substr = substr.strip()
        result[substr] = len(substr)
    return result

In [16]:
s = "Ask not what your country can do for you, but what you can do for your country"

result = longest_common_substring(s)
result = sorted(result.items(), key=lambda x: x[1], reverse=True)
result[:5]

[('can do for you', 14),
 ('an do for you', 13),
 ('your country', 12),
 ('n do for you', 12),
 ('our country', 11)]

## 15.3 原理

解决字符串问题的技巧：
- 字符串的数据结构
- 散列：平均速度很快，且易于实现。
- 平衡树：在最坏情况下也有较好的性能。
- 后缀数组：初始化指向文本中每个字符（或每个单词）的指针数组，对其排序就得到一个后缀数组，遍历该数组以查找接近的字符串。